Before running following cell, please create file users.py and create a list with Twitter user nicknames in this file, e.g.: 

users = ['aaa', 'bbb', 'ccc']

In [37]:
import twitter_scraper
from twitter_scraper import get_tweets_search
import pandas as pd
import csv
import xml
from lxml.etree import ParserError
import os, path, sys
import glob

import re
import matplotlib.dates
import matplotlib
import matplotlib.pyplot
import datetime
import users

In [35]:
url_cache = dict() # twitter_scraper.load_url_cache("/tmp/web-cache_compact.dat")
last_request_time = katya_twitter_scraper.Bag(0)

In [ ]:
for user in users:
    pages = 4500 # 20 tweets per page
    l = []
    for tweet in get_tweets_search(user, url_cache, last_request_time, pages):
        l.append(tweet)
    print("Downloaded {} tweets for user {}".format(len(l), user))
    print(datetime.datetime.now())
    df = pd.DataFrame(l, columns = tweet.keys())
    df.to_csv('users/{}.csv'.format(user))

In [ ]:
twitter_scraper.save_url_cache(url_cache, "/tmp/web-cache_compact.dat")

In [2]:
path ='users//' 
allFiles = glob.glob(path + "/*.csv")

list_ = []

for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0,
                 lineterminator='\n')
    list_.append(df)

df = pd.concat(list_, axis = 0, ignore_index = True)

df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [3]:
len(df)

1829451

In [ ]:
df.to_csv('users/df.csv')

Text Preprocessing

In [ ]:
df = pd.read_csv('users/df.csv', index_col=0, header=0,
                 lineterminator='\n')

In [1]:
def twi_replace(text):
    if 'pic.twitter' in text:
        new = re.sub('pic.twitter.com/[\w]+', ' urlTag ', text)
        return new
    else:
        return text

In [ ]:
twi_replace('Ось тримай пательню, щоб жар не псувати. Вьо нахт кухен, як пропонує попередній оратор.pic.twitter.com/30xKIH2FlY')

In [ ]:
from pynlple.processing.preprocessor import *

stack = [
                                        HtmlEntitiesUnescaper(),
                                        BoldTagReplacer(),
                                        URLReplacer(' urlTag '),
                                        HtmlTagReplacer(' '),
                                        EmailReplacer(' emailTag '),
                                        PhoneNumberReplacer(' phoneNumberTag '),
                                        AtReferenceReplacer(' atRefTag '),
                                        CommaReplacer(),
                                        QuotesReplacer(),
                                        DoubleQuotesReplacer(),
                                        SoftHyphenReplacer(),
                                        DashAndMinusReplacer(),
                                        TripledotReplacer(),
                                        ToLowerer(),
                                        DigitsReplacer(True, '0'),
                                        MultiPunctuationReplacer(),
                                        MultiNewLineReplacer(),
                                        MultiWhitespaceReplacer(),
                                        WordTokenizer(),
                                        Trimmer(),
        ]

In [ ]:
%time
t = StackingPreprocessor(stack)
df['preproc_text'] = df["text"].apply(lambda x: t.preprocess(twi_replace(str(x)))).values

Language Detection

In [ ]:
import pycld2 as cld2
%time
df['cld2'] = [cld2.detect(str(i))[2][0][1] for i in df['preproc_text']]

In [ ]:
from pyfasttext import FastText
model = FastText('models/lid.176.bin') 
df['ft'] = [model.predict_proba([str(i)], 1)[0][0][0] for i in df['preproc_text']]

In [ ]:
df.groupby('cld2').count()

In [ ]:
df.groupby('ft').count()